## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**1270. All People Report to the Given Manager (Medium)**

**Table: Employees**

| Column Name   | Type    |
|---------------|---------|
| employee_id   | int     |
| employee_name | varchar |
| manager_id    | int     |

employee_id is the column of unique values for this table.
Each row of this table indicates that the employee with ID employee_id and name employee_name reports his work to his/her direct manager with manager_id
The head of the company is the employee with employee_id = 1.
 
**Write a solution to find employee_id of all employees that directly or indirectly report their work to the head of the company.**

The indirect relation between managers will not exceed three managers as the company is small.

Return the result table in any order.

The result format is in the following example.

**Example 1:**

**Input:** 

**Employees table:**

| employee_id | employee_name | manager_id |
|-------------|---------------|------------|
| 1           | Boss          | 1          |
| 3           | Alice         | 3          |
| 2           | Bob           | 1          |
| 4           | Daniel        | 2          |
| 7           | Luis          | 4          |
| 8           | Jhon          | 3          |
| 9           | Angela        | 8          |
| 77          | Robert        | 1          |

**Output:** 
| employee_id |
|-------------|
| 2           |
| 77          |
| 4           |
| 7           |

**Explanation:** 
- The head of the company is the employee with employee_id 1.
- The employees with employee_id 2 and 77 report their work directly to the head of the company.
- The employee with employee_id 4 reports their work indirectly to the head of the company 4 --> 2 --> 1. 
- The employee with employee_id 7 reports their work indirectly to the head of the company 7 --> 4 --> 2 --> 1.
- The employees with employee_id 3, 8, and 9 do not report their work to the head of the company directly or indirectly.

In [0]:
employees_data_1270 = [
    (1, "Boss", 1), 
    (3, "Alice", 3), 
    (2, "Bob", 1), 
    (4, "Daniel", 2), 
    (7, "Luis", 4), 
    (8, "Jhon", 3), 
    (9, "Angela", 8), 
    (77, "Robert", 1)
]
employees_columns_1270 = ["employee_id", "employee_name", "manager_id"]
employees_df_1270 = spark.createDataFrame(employees_data_1270, employees_columns_1270)
employees_df_1270.show()

In [0]:
level_1_1270 = employees_df_1270\
                    .filter(col("manager_id") == 1).select("employee_id")

In [0]:
level_2_1270 = employees_df_1270.alias("e1") \
    .join(level_1_1270.alias("l1"), col("e1.manager_id") == col("l1.employee_id"), how="inner") \
    .select(col("e1.employee_id").alias("emp_id"))

In [0]:
level_3_1270 = employees_df_1270.alias("e2") \
    .join(level_2_1270.alias("l2"), col("e2.manager_id") == col("l2.emp_id"), how="inner") \
    .select(col("e2.employee_id").alias("emp_id"))

In [0]:
result_df_1270 = level_1_1270.union(level_2_1270).union(level_3_1270).distinct()

In [0]:
result_df_1270\
    .filter(col("employee_id") != 1)\
        .orderBy("employee_id").show()